In [ ]:
# only for cuda enabled laptop and desktop
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

import os

for fname in os.listdir('My_Data/train/Chetan'):
  print(fname)

In [ ]:
for fname in os.listdir('My_Data/train/Sanket'):
  print(fname)

In [ ]:
image_1 = load_img('My_Data/train/Sanket/sanket_1.jpg')
image_1

In [ ]:
path = os.path.join('My_Data')
train_dir = os.path.join(path,'train')
validation_dir = os.path.join(path,'validation')
train_chetan_dir = os.path.join(train_dir,'Chetan')
train_sanket_dir = os.path.join(train_dir,'Sanket')
validation_chetan_dir = os.path.join(validation_dir,'Chetan')
validation_sanket_dir = os.path.join(validation_dir,'Sanket')

In [ ]:
num_chetan_tr = len(os.listdir(train_chetan_dir))
num_sanket_tr = len(os.listdir(train_sanket_dir))
num_chetan_val = len(os.listdir(validation_chetan_dir))
num_sanket_val = len(os.listdir(validation_sanket_dir))

In [ ]:
total_train=num_chetan_tr+num_sanket_tr
total_val=num_chetan_val+num_sanket_val
print(total_train)
print(total_val)

In [ ]:
epochs = 50
IMG_HIEGHT = 150
IMG_WIDHT = 150

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                    rotation_range=45,
                                    width_shift_range=.15,
                                    height_shift_range=.15,
                                    horizontal_flip=True,
                                     vertical_flip=True,
                                     brightness_range=[0.5,1.5],
                                    zoom_range=0.5)

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(directory=train_dir,
                                                    shuffle=True,
                                                    target_size=(IMG_HIEGHT,IMG_HIEGHT),
                                                    class_mode='binary')

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1,7,figsize=(20,20))
    axes = axes.flatten()
    for img,ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(7)]
plotImages(augmented_images)

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(directory=validation_dir,
                                                target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                class_mode='binary')

In [ ]:
model = Sequential([
    Conv2D(16,3,padding='same',activation='relu',input_shape=(IMG_HIEGHT,IMG_WIDHT,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(256,activation='relu'),
    Dense(128,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_data_gen,epochs=epochs,validation_data=val_data_gen)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Val Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Val LOss')
plt.legend(loc='lower right')
plt.title('Loss')
plt.show()

In [ ]:
data_1 = load_img('My_Data/train/Sanket/sanket_13.jpg',target_size=(IMG_HIEGHT,IMG_WIDHT))
data_1

In [ ]:
data_2 = img_to_array(data_1)
img_test = data_2.reshape(1,150,150,3)

In [ ]:
pred = model.predict(img_test)
int(pred[0][0])

In [ ]:
class_name = ['Chetan','Sanket']
#testing cascad
import cv2
import numpy as np
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')
while True:
    # Read the frame
    _, img = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    test = img
    test = cv2.resize(test,(150,150))
    test = test.reshape(1,150,150,3)
    iden = []
    pred = model.predict(test)
    iden.append(int(pred[0][0]))
    print(iden)
    

    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    #print(faces.shape)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(img, str(class_name[int(pred[0][0])]), (x+5,y-5), font, 1, (255,0,0), 4)

    # Display
    cv2.imshow('img', img)

    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()